In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))

In [3]:
net(X)

tensor([[0.0210],
        [0.0274]], grad_fn=<AddmmBackward0>)

In [4]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2207, -0.1578,  0.1179,  0.0350,  0.0699, -0.0762, -0.3345,  0.0511]])), ('bias', tensor([-0.0105]))])


In [5]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0105], requires_grad=True)
tensor([-0.0105])


In [6]:
net[2].weight.grad is None

True

In [7]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [8]:
net.state_dict()['2.bias'].data

tensor([-0.0105])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2263],
        [-0.2265]], grad_fn=<AddmmBackward0>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
print(rgnet[0][1][0].bias)

Parameter containing:
tensor([-0.1716,  0.1770,  0.4381,  0.4393, -0.0346, -0.1580,  0.4669,  0.1933],
       requires_grad=True)


In [12]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0047,  0.0236,  0.0161, -0.0105]), tensor(0.))

In [13]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [14]:
def init_xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[ 0.0308,  0.3335,  0.0044,  0.6225],
        [-0.2116, -0.1307,  0.4711,  0.0208],
        [-0.6415, -0.3432,  0.2589, -0.3614],
        [ 0.6079, -0.1679,  0.3192, -0.4149],
        [ 0.1699,  0.4158,  0.3936, -0.2877],
        [-0.0146, -0.6347, -0.6899,  0.6897],
        [ 0.3128,  0.6089, -0.2773,  0.3218],
        [-0.6880, -0.4260,  0.3477,  0.1251]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [17]:
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -0.0000, -8.7441, -7.0742],
        [ 5.3636, -8.9511, -8.9778,  5.7546]], grad_fn=<SliceBackward0>)

In [18]:
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -7.7441, -6.0742])

In [19]:
shared=nn.Linear(8,8)

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))

net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])

net[2].weight.data[0,0]=100

print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [20]:
def get_net(input_feature,output_feature):
    net=nn.Sequential(nn.Linear(input_feature,8),nn.ReLU(),nn.Linear(8,output_feature))
    return net

net=get_net(4,1)
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
